# Data Collection

In [15]:
import csv
from docx import Document

def read_table_from_docx(docx_file_path):
    """
    Extracts only table from DOCX file and returns its data
    """
    document = Document(docx_file_path)
    table = document.tables[0]
    table_data = []
    for row in table.rows:
            row_data = [cell.text.replace('\xa0', '').rstrip() for cell in row.cells]
            table_data.append(row_data)
    return table_data

def save_table_to_csv(table_data, csv_file_path):
    """
    Saves table data to a CSV file.
    """
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(table_data)

In [19]:
import os

docx_file_path = '..\data_raw\Assignment 1.docx'
csv_file_path = '..\data_raw\question_1.csv'

# Check if CSV file already exists
if (not os.path.isfile(csv_file_path)):
    # Extract table from DOCX file and save to CSV
    table = read_table_from_docx(docx_file_path)
    save_table_to_csv(table, csv_file_path)

In [20]:
import pandas as pd

# Load data from CSV file
data = pd.read_csv(csv_file_path)
print(data)

   Height  Weight  Age  Grip strength Frailty
0    65.8     112   30             30       N
1    71.5     136   19             31       N
2    69.4     153   45             29       N
3    68.2     142   22             28       Y
4    67.8     144   29             24       Y
5    68.7     123   50             26       N
6    69.8     141   51             22       Y
7    70.1     136   23             20       Y
8    67.9     112   17             19       N
9    66.8     120   39             31       N


C:\Users\rgbmr\AppData\Local\Temp\ipykernel_10340\4238834641.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
document = Document(docx_file_path)